## dCMF
Example of running the "dcmf" module with the use provided parameters

In [1]:
import sys
sys.path.append("..")

In [2]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os

In [3]:
from src.dcmf import dcmf

## Loading the sample dataset

This directory contains a sample synthetic dataset generated for the augmented setting of Fig 1(c) in the [paper](https://arxiv.org/abs/1811.11427).
You can download the sample data from [here](https://drive.google.com/open?id=1EFF_kuOIg2aYyOGZY_peX3NziqCSxxP1) and unzip it to the data directory.

In [4]:
sample_no = 4
data_dir = f"../data/MIMIC/sample{sample_no}/"

In [5]:
#Loads the dataset into a dict
#Note: This dataset contains 5-folds for the matrix X_12 (matrix R below)
num_folds = 1
#
pp = pprint.PrettyPrinter()
print("Loading data from data_dir: ",data_dir)
U1 = pkl.load(open(data_dir+"X_12.pkl",'rb'))
V1 = pkl.load(open(data_dir+"X_31.pkl",'rb'))
# V1 = pkl.load(open(data_dir+"X_26.pkl",'rb'))
# W1 = pkl.load(open(data_dir+"X_53.pkl",'rb'))
R_temp_dict = {}
for fold_num in np.arange(1,num_folds+1):
    Rtrain = pkl.load(open(data_dir+'/X_23_train_fold_'+str(fold_num)+'.pkl','rb'))
    Rtrain = Rtrain
    Rtrain_idx = pkl.load(open(data_dir+'/X_23_train_idx_'+str(fold_num)+'.pkl','rb')) 
    Rtest = pkl.load(open(data_dir+'/X_23_test_fold_'+str(fold_num)+'.pkl','rb'))
    Rtest_idx = pkl.load(open(data_dir+'/X_23_test_idx_'+str(fold_num)+'.pkl','rb'))
    Rdoublets = pkl.load(open(data_dir+'/R_doublets_'+str(fold_num)+'.pkl','rb'))
    R_temp_dict[fold_num] = {"Rtrain":Rtrain,"Rtrain_idx":Rtrain_idx,"Rtest":Rtest,"Rtest_idx":Rtest_idx,"Rdoublets":Rdoublets}
#
data_dict = {"U1":U1,"V1":V1,"R":R_temp_dict}

Loading data from data_dir:  ../data/MIMIC/sample4/


In [6]:
print("U1.shape: ",U1.shape)
print("V1.shape: ",V1.shape)
print("R.shape: ",data_dict['R'][1]['Rtrain'].shape)

U1.shape:  (5911, 1321)
V1.shape:  (596, 5911)
R.shape:  (1321, 596)


## Building the required data structures

Here we construct the data structures required as input to the dcmf API

#### *entity matrix relationship graph *

- **G**: dict, keys are entity IDs and values are lists of associated matrix IDs

#### * training data*
- **X_data**: dict, keys are matrix IDs and values are (1) np.array, or (2) dict, (if this matrix is in validation set **X_val**) with validation set IDs as keys & values as np.array
- **X_meta**: dict, keys are matrix IDs and values are lists of the 2 associated entity IDs

#### *validation data*
- **X_val**: dict, keys are IDs of the matrices that are part of validation set and values are dict with validation set IDs as keys and values are (1) scipy.sparse matrix, or (2) list of triplets corresponding to the validation entries (if you would like to perform classification and measure AUC)  
**Note**: To perform K folds cross validation, use K validation sets for the corresponsing matrix/matrices. In the example below, we used a single validation set with ID "1" for each of the matrices with IDs "X1" and "X2"

In [7]:
G = {
    "e1":["X1","X3"],\
    "e2":["X1","X2"],\
    "e3":["X2","X3"]
}
    #"e6":["X4"]}

In [8]:
X_data = {
    "X1":{"1":U1},\
    "X2":{"1":data_dict['R'][1]["Rtrain"]},\
    "X3":V1}

In [9]:
X_meta = {
    "X1":["e1","e2"],\
    "X2":["e2","e3"],\
    "X3":["e3","e1"]}
    #"X5":["e5","e3"]}

In [10]:
# Rtest_triplets1 = [[1,1,1],[2,2,0]]
# Rtest_triplets2 = [[1,1,1],[3,3,0],[1,2,0],[0,1,0],[0,2,0],[0,3,0]]
Rtest_triplets1 = [[0, 65, 1], [0, 3, 0]]
Rtest_triplets2 = [[0, 64, 1], [0, 66, 0]]

In [11]:
X_val = {
    "X1":{"1":Rtest_triplets1},
    "X2":{"1":Rtest_triplets2}
}

#### *dCMF network construction - hyperparameters*

- **kf**: float, in the range (0,1) 
- **k**: int, entity representation or encoding size. Refer Appendix A in the [paper](https://arxiv.org/abs/1811.11427) for info about how k and kf are used in the dCMF network construction. 
- **e_actf**: str, autoencoder's encoding activation function.
- **d_actf**: str, autoencoder's decoding activation function. Supported functions are "tanh","sigma","relu","lrelu"
- **is_linear_last_enc_layer**: bool, True to set linear activation for the bottleneck/encoding generation layer 
- **is_linear_last_dec_layer**: bool, True to set linear activation for the output/decoding generation layer 
- **num_chunks**: int, number of training batches to create.

In [12]:
kf = 0.5
k = 100
e_actf = "tanh"
d_actf = "tanh"
is_linear_last_enc_layer = False
is_linear_last_dec_layer = False
num_chunks = 2

#### *Optimization/training - hyperparamteres*

- **learning_rate**: float, Adam optimizer's learning rate
- **weight_decay**: float, Adam optimizers's weight decay (L2 penalty)
- **max_epochs**: int, maximum number of training epochs at which the training stops 
- **convg_thres**: float, convergence threshold 

In [13]:
learning_rate = 0.001
weight_decay = 0.05
max_epochs = 5
convg_thres = 0.1

#### *Hyperparamteres related to pre-training*

- **is_pretrain**: bool, True for pretraining 
- **pretrain_thres**: bool, pre-training convergence thresholsd
- **max_pretrain_epochs**: int, maximum number of pre-training epochs at which the training stops

In [14]:
is_pretrain=True
pretrain_thres= 0.1
max_pretrain_epochs = 2

#### *Parameters related to validation*

- **val_metric**: str, Validation performance metric. Supported metrics: ["rmse","r@k","p@k","auc"]. Where,  
     *rmse* - Root [mean square error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)  
     *r@k* - Recall@k. Refer section 5.2's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *p@k* - Probability@k. Refer section 5.3's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *auc* - [Area under the curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    
- **is_val_transpose**: bool, True if the reconstructed matrix has to be transposed before computing the validation performance
- **at_k**: int, the value of k if the **val_metric** is either "r@k" or "p@k"

In [15]:
val_metric = "auc"
is_val_transpose = True
at_k = 10

#### *GPU - parameters *

- **is_gpu**: bool, True if pytorch tensors storage and operations has to be done in GPU
- **gpu_ids**: str, Comma separated string of CUDA GPU ID

In [16]:
is_gpu = False
gpu_ids = "1"

#### *Instantiating the dCMF model...*
- Initializes dCMF after validating the input data and the (hyper)parameters

In [17]:
dcmf_model = dcmf(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,X_val=X_val,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)

dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF:
---
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  3
num matrices:  3
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#


#### *Fitting... *
- Performs the input transformation and network construction
- (Pre-trains and) trains the model to obtain the entity representations
- Reconstruct the input matrices using the entity representations obtained

In [18]:
dcmf_model.fit()

## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  3
num matrices:  3
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
dcmf - model construction - start
__input_transformation - start
#
concatenated-matrix construction...
e_id:  e1
X_id_list:  ['X1', 'X3']
X_id:  X1
X[X_id].shape:  (5911, 1321)
X_id:  X3
X[X_id].shape:  (596, 5911)
C_dict[e].shape:  torch.Size([5911, 1917])
---
e_id:  e2
X_id_list:  ['X1', 'X2']
X_id:  X1
X[X_id].shape:  (5911, 1321)
X_id:  X2
X[X_id].shape:  (1321, 596)
C_dict[e].shape:  torch.Size(

#### *Result attributes:*
- **out_dict_U**:  dict, keys are validation set IDs and values are dict with entity IDs as keys and np.array of entity representations/encodings as values
- **out_dict_X_prime**: dict, keys are matrix IDs and values are matrix reconstructions
- **out_dict_info**: dict, keys are loss/validation performance attributes and values are corresponding results.

In [19]:
dcmf_model.out_dict_U['1'].keys()

dict_keys(['e1', 'e2', 'e3'])

In [20]:
dcmf_model.out_dict_X_prime['1'].keys()

dict_keys(['X1', 'X2', 'X3'])

In [21]:
dcmf_model.out_dict_info

{'params': {'learning_rate': 0.001,
  'weight_decay': 0.05,
  'convg_thres': 0.1,
  'max_epochs': 5,
  'is_pretrain': True,
  'pretrain_thres': 0.1,
  'max_pretrain_epochs': 2,
  'num_chunks': 2,
  'k': 100,
  'kf': 0.5,
  'e_actf': 'tanh',
  'd_actf': 'tanh',
  'is_linear_last_enc_layer': False,
  'is_linear_last_dec_layer': False},
 'num_val_sets': 1,
 'loss_all_folds': {'1': [0.02807197067886591,
   0.07087323069572449,
   0.187986321747303,
   0.007273016730323434,
   0.41838957369327545,
   0.06768366694450378]},
 'loss_all_folds_avg_tuple': [0.02807197067886591,
  0.07087323069572449,
  0.187986321747303,
  0.007273016730323434,
  0.41838957369327545,
  0.06768366694450378],
 'loss_all_folds_avg_sum': 0.7802777804899961,
 'val_metric': 'auc',
 'val_perf_all_folds': {'1': {'X1': 0.0, 'X2': 0.0}},
 'val_perf_all_folds_avg': {'X1': 0.0, 'X2': 0.0},
 'val_perf_all_folds_total': {'1': 0.0},
 'val_perf_all_folds_total_avg': 0.0,
 'E': 3,
 'M': 3}

In [22]:
patient_emb = list(dcmf_model.out_dict_U['1'].values())[0].detach().numpy()
disease_emb = list(dcmf_model.out_dict_U['1'].values())[1].detach().numpy()
drug_emb = list(dcmf_model.out_dict_U['1'].values())[2].detach().numpy()

In [23]:
import pandas as pd

In [24]:
patient_emb_df = pd.DataFrame(patient_emb)
patient_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.003116,0.042935,0.053690,0.095826,0.010394,-0.002317,0.054300,-0.028238,-0.004080,-0.049606,...,-0.038036,0.034357,0.063154,0.061753,0.088627,0.000102,0.077342,-0.002649,0.079266,0.008935
1,-0.002405,0.044990,0.055435,0.096376,0.009969,-0.000587,0.057421,-0.029114,-0.006704,-0.046754,...,-0.038259,0.034903,0.063888,0.061920,0.089260,0.002405,0.075944,0.001103,0.079968,0.006919
2,-0.001896,0.042640,0.054686,0.096711,0.009165,-0.003492,0.054875,-0.029136,-0.005436,-0.049697,...,-0.039336,0.035139,0.062594,0.061336,0.088494,0.000219,0.078663,-0.002022,0.079234,0.008409
3,-0.001191,0.043862,0.055083,0.097234,0.008968,-0.001827,0.054547,-0.028639,-0.003009,-0.049880,...,-0.039982,0.035260,0.065551,0.063094,0.088260,0.001949,0.078456,-0.002426,0.078826,0.007841
4,-0.002614,0.043560,0.054303,0.098030,0.009182,-0.000323,0.054201,-0.028288,-0.003900,-0.048991,...,-0.039248,0.037008,0.064502,0.063143,0.088151,0.002409,0.077926,-0.003148,0.080253,0.007153


In [25]:
patient_emb_df.shape

(5911, 100)

In [26]:
disease_emb_df = pd.DataFrame(disease_emb)
disease_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.014093,-0.047882,0.092390,0.099943,-0.074168,0.052060,0.080090,0.021332,-0.000255,0.017155,...,-0.030044,-0.127665,-0.024481,-0.034066,0.003162,-0.023032,-0.050521,-0.040707,0.092297,0.030234
1,0.017447,-0.045624,0.091778,0.092480,-0.079177,0.041846,0.081640,0.014429,0.002047,0.023557,...,-0.024132,-0.116535,-0.029018,-0.024768,0.002858,-0.012564,-0.042414,-0.044521,0.081603,0.040826
2,0.017035,-0.045878,0.091906,0.093344,-0.078686,0.042992,0.081515,0.015157,0.001740,0.022737,...,-0.024811,-0.117990,-0.028352,-0.025958,0.002887,-0.013845,-0.043386,-0.043979,0.082885,0.039573
3,0.011115,-0.049749,0.092818,0.106393,-0.069750,0.060975,0.078798,0.027255,-0.002288,0.011671,...,-0.035229,-0.137342,-0.020517,-0.041961,0.003406,-0.032121,-0.057587,-0.037306,0.101559,0.021001
4,0.018333,-0.045093,0.091629,0.090610,-0.080483,0.039343,0.082051,0.012664,0.002672,0.025178,...,-0.022528,-0.113794,-0.030241,-0.022452,0.002763,-0.010134,-0.040426,-0.045428,0.078973,0.043489


In [27]:
disease_emb_df.shape

(1321, 100)

In [28]:
drug_emb_df = pd.DataFrame(drug_emb)
drug_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.061985,-0.080801,0.012557,0.089628,-0.019293,0.055401,0.094996,0.021540,-0.024506,-0.021499,...,-0.085503,-0.111388,-0.037245,-0.016952,0.070906,-0.005747,-0.033870,0.068399,0.105988,-0.089275
1,-0.078831,-0.094036,0.029251,0.111642,-0.038221,0.071943,0.104433,0.028042,-0.041212,-0.017292,...,-0.087705,-0.144672,-0.032274,-0.015788,0.081794,0.008261,-0.055025,0.069337,0.119441,-0.099637
2,-0.082099,-0.096387,0.032347,0.116039,-0.041661,0.074954,0.106350,0.029257,-0.044435,-0.016447,...,-0.088090,-0.150883,-0.031251,-0.015440,0.083925,0.010872,-0.059041,0.069629,0.122105,-0.101702
3,-0.067145,-0.084992,0.017696,0.096156,-0.024824,0.060600,0.097621,0.023562,-0.029543,-0.020060,...,-0.086161,-0.121429,-0.035609,-0.016658,0.074005,-0.001429,-0.040456,0.068403,0.109886,-0.092269
4,-0.066525,-0.084537,0.017240,0.095527,-0.024146,0.060102,0.097306,0.023427,-0.028843,-0.020232,...,-0.086108,-0.120274,-0.035765,-0.016785,0.073679,-0.001913,-0.039837,0.068335,0.109438,-0.091941


In [29]:
drug_emb_df.shape

(596, 100)

In [30]:
emb_df = pd.concat([patient_emb_df, disease_emb_df, drug_emb_df], ignore_index = True, axis = 0)
emb_df.shape

(7828, 100)

In [31]:
emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.003116,0.042935,0.053690,0.095826,0.010394,-0.002317,0.054300,-0.028238,-0.004080,-0.049606,...,-0.038036,0.034357,0.063154,0.061753,0.088627,0.000102,0.077342,-0.002649,0.079266,0.008935
1,-0.002405,0.044990,0.055435,0.096376,0.009969,-0.000587,0.057421,-0.029114,-0.006704,-0.046754,...,-0.038259,0.034903,0.063888,0.061920,0.089260,0.002405,0.075944,0.001103,0.079968,0.006919
2,-0.001896,0.042640,0.054686,0.096711,0.009165,-0.003492,0.054875,-0.029136,-0.005436,-0.049697,...,-0.039336,0.035139,0.062594,0.061336,0.088494,0.000219,0.078663,-0.002022,0.079234,0.008409
3,-0.001191,0.043862,0.055083,0.097234,0.008968,-0.001827,0.054547,-0.028639,-0.003009,-0.049880,...,-0.039982,0.035260,0.065551,0.063094,0.088260,0.001949,0.078456,-0.002426,0.078826,0.007841
4,-0.002614,0.043560,0.054303,0.098030,0.009182,-0.000323,0.054201,-0.028288,-0.003900,-0.048991,...,-0.039248,0.037008,0.064502,0.063143,0.088151,0.002409,0.077926,-0.003148,0.080253,0.007153


In [32]:
with open(f"emb_MIMIC_sample_{sample_no}.dat", "w") as file:
    file.write("\n")
    for idx, row in emb_df.iterrows():
        emb = row[:].astype(np.float32)
        emb_str = ' '.join(emb.astype(str))
        file.write(f'{idx}\t{emb_str}\n')
    